<a href="https://colab.research.google.com/github/MohammadSarfaraz/Data-Analytics/blob/main/gee_loc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment-2 


### The task is related to geospatial data science.

1. Find out the list of open source(free to use) remote sensing satellites that offers the data.
2. Obtain the imagery data of your location, any of the service. eg copernicus  
3. The output of your result is the final image of your location (or any location given by you).
4. No further analysis is needed.


# Earth Engine Python API Colab Setup

This notebook demonstrates how to setup the Earth Engine Python API in Colab and provides several examples of how to print and visualize Earth Engine processed data.


### using cmd command-:
* python -m ipykernel install --user --name GEE

* conda install -c conda-forge earthengine-api



## Import API and get credentials

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [ ]:
import ee

In [ ]:
print(ee)

<module 'ee' from 'C:\\Users\\hp\\Anaconda3\\lib\\site-packages\\ee\\__init__.py'>


### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [ ]:
# Trigger the authentication flow.
#ee.Authenticate()


### Test the API

Test the API by printing the elevation of Mount Everest.

In [ ]:
from IPython.display import Image
ee.Initialize()

# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


## Map visualization

`ee.Image` objects can be displayed to notebook output cells. The following two
examples demonstrate displaying a static image and an interactive map.


### Static image

The `IPython.display` module contains the `Image` function, which can display
the results of a URL representing an image generated from a call to the Earth
Engine `getThumbUrl` function. The following cell will display a thumbnail
of the global elevation model.

In [ ]:
# Import the Image function from the IPython.display module. 
from IPython.display import Image

# Display a thumbnail of global elevation.
Image(url = dem.updateMask(dem.gt(0))
  .getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

### Interactive map

The [`folium`](https://python-visualization.github.io/folium/)
library can be used to display `ee.Image` objects on an interactive
[Leaflet](https://leafletjs.com/) map. Folium has no default
method for handling tiles from Earth Engine, so one must be defined
and added to the `folium.Map` module before use.

The following cell provides an example of adding a method for handing Earth Engine
tiles and using it to display an elevation model to a Leaflet map.

In [ ]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

## Chart visualization

Some Earth Engine functions produce tabular data that can be plotted by
data visualization packages such as `matplotlib`. The following example
demonstrates the display of tabular data from Earth Engine as a scatter
plot. See [Charting in Colaboratory](https://colab.sandbox.google.com/notebooks/charts.ipynb)
for more information.

In [ ]:
# Import the matplotlib.pyplot module.
import matplotlib.pyplot as plt

# Fetch a Landsat image.
img = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_034033_20000913')

# Select Red and NIR bands, scale them, and sample 500 points.
samp_fc = img.select(['B3','B4']).divide(10000).sample(scale=30, numPixels=500)

# Arrange the sample as a list of lists.
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(2), ['B3', 'B4'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

# Display a scatter plot of Red-NIR sample pairs using matplotlib.
plt.scatter(samp_data[0], samp_data[1], alpha=0.2)
plt.xlabel('Red', fontsize=12)
plt.ylabel('NIR', fontsize=12)
plt.show()

<Figure size 640x480 with 1 Axes>

## Extracting data on Total Annual Rainfall and Maximum EVI for a given year 

In [ ]:
import ee
import pandas as pd

ee.Initialize()

# get states of India as a feature collection
# states = ee.FeatureCollection('users/mdm/india_soiStates')
districts = ee.FeatureCollection('users/mdm/za/za_districts')

# Obtain daily rainfall data for 2018 and add it up to get tot. ann. rainfall
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")\
.filterDate('2018-01-01', '2018-12-31')\
.sum()

# Get MODIS 16-day Veg Indices dataset, select EVI for 2018, get median value
evi = ee.ImageCollection('MODIS/006/MOD13A2')\
.select('EVI')\
.filterDate('2018-01-01', '2018-12-31')\
.max()\
.divide(10000)

# Create a composite image with rainfall and evi as bands
composite = ee.Image.cat([chirps, evi])

# Extract median value of rainfall and evi for each state at a 5 km scale
dist_stats = composite.reduceRegions(districts, 'median', 5000)

# Rename feature properties, dump geometries and get dictionary
dist_stats = dist_stats.select(['DISTRICT_N', 'precipitation', 'EVI'],
                                 ['district', 'tot_rainfall', 'med_evi'],
                                 retainGeometry=False).getInfo()

# Convert dictionary into pandas dataframe
dtstats_df = pd.DataFrame()
for dist in dist_stats['features']:
    df = pd.DataFrame([dist['properties']], columns=dist['properties'].keys())
    dtstats_df = pd.concat([dtstats_df, df], axis=0)

# Reorder columns in dataframe, sort by state name and reset index
dtstats_df = dtstats_df[['district', 'tot_rainfall', 'med_evi']]
dtstats_df = dtstats_df.sort_values(by=['district']).reset_index(drop=True)

# change state names from upper case to title case!!
dtstats_df['district'] = dtstats_df['district'].str.title()

# display dataframe
dtstats_df


# # Export pandas dataframe to CSV
# dtstats_df.to_csv('./dist_stats.csv', index=False)

,district,tot_rainfall,med_evi
0,Alfred Nzo,785.515781,0.387722
1,Amajuba,675.467849,0.448054
2,Amathole,650.129219,0.420122
3,Bojanala,490.826156,0.358479
4,Buffalo City,620.396744,0.450135
5,Cacadu,347.632161,0.204925
6,Cape Winelands,269.560990,0.150458
7,Capricorn,387.629911,0.307675
8,Central Karoo,188.979677,0.099214
9,Chris Hani,497.791215,0.292200


## Some actual EE code in Python! 


In [ ]:


poi = ee.Geometry.Point([76.5384, 11.7253])


s2 = ee.ImageCollection('COPERNICUS/S2').filterBounds(poi).filterDate('2019-02-20', '2019-02-28').min()

# Specify visualisation parameters for Natural Colour and Short-wave Infrared images
s2_rgb = ee.Image(s2).visualize(min=300, max=2800, bands=['B4', 'B3', 'B2'])
s2_swir = ee.Image(s2).visualize(min=500, max=3300, bands=['B12', 'B8', 'B5'])

# Create a folium basemap into which EE layers above will be added
myMap = folium.Map(location=[11.70, 76.584], zoom_start=12, tiles='Stamen Terrain', control_scale=True, detect_retina=True)


# Add a layer control widget to the map
folium.LayerControl().add_to(myMap)



# Show the map!
myMap